Students implement Hill-Climbing Search, Local Beam Search, and Simulated Annealing Search algorithms following TODO 1 - 3. \
Students can add supporting attributes and methods to the three classes as needed.

In [19]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Libraries

In [1]:
import os
import heapq
import random
import math

# Graph class

In [40]:
# Directed, weighted graphs
class Graph:
  def __init__(self):
    self.AL = dict() # adjacency list
    self.V = 0
    self.E = 0
    self.H = dict()

  def __str__(self):
    res = 'V: %d, E: %d\n'%(self.V, self.E)
    for u, neighbors in self.AL.items():
      line = '%d: %s\n'%(u, str(neighbors))
      res += line
    for u, h in self.H.items():
      line = 'h(%d) = %d\n'%(u, h)
    return res

  def print(self):
    print(str(self))

  def load_from_file(self, filename):
    # '''
    #     Example input file:
    #         V E
    #         u v w
    #         u v w
    #         u v w
    #         ...
    #         u1 h1
    #         u2 h2
    #         u3 h3
    #         ...

    #     # input.txt
    #     7 8
    #     0 1 5
    #     0 2 6
    #     1 3 12
    #     1 4 9
    #     2 5 5
    #     3 5 8
    #     3 6 7
    #     4 6 4
    #     0 14
    #     1 13
    #     2 12
    #     3 11
    #     4 10
    #     5 9
    #     6 0
    # '''
    if os.path.exists(filename):
      with open(filename) as g:
        self.V, self.E = [int(it) for it in g.readline().split()]
        for i in range(self.E):
          line = g.readline()
          u, v, w = [int(it) for it in line.strip().split()]
          if u not in self.AL:
            self.AL[u] = []
          self.AL[u].append((v, w))
        for i in range(self.V):
          line = g.readline()
          u, h = [int(it) for it in line.strip().split()]
          self.H[u] = h

In [41]:
g = Graph()
g.load_from_file('/content/drive/MyDrive/Colab_Notebooks/input.txt') #Path to input.txt
g.print()

V: 7, E: 8
0: [(1, 5), (2, 6)]
1: [(3, 12), (4, 9)]
2: [(5, 5)]
3: [(5, 8), (6, 7)]
4: [(6, 4)]



# Search Strategies

In [42]:
class LocalSearchStrategy:
  def search(self, g: Graph, src: int) -> tuple:
        visited = set()
        queue = [(src, 0)]
        best_state = None
        best_priority = float('-inf')
        while queue:
            u, p = queue.pop(0)
            if u in visited:
                continue
            visited.add(u)
            # Update the best state if necessary
            if p > best_priority:
                best_state = u
                best_priority = p
            # Explore the neighbors of u
            if u in g.AL:
                for v, w in g.AL[u]:
                    if v not in visited:
                        queue.append((v, p + w + g.H[v]))
        return (best_state, best_priority)
    # '''
    # return a tuple (u, p) in which
    #   u: the local maximum state
    #   p: the priority/weight/desirability/cost of u
    # '''
    # return (None, None)

In [43]:
class HillClimbingSearch(LocalSearchStrategy):
  def search(self, g: Graph, src: int) -> tuple:
    visited = set()
    stack = [(src, 0)]
    best_state = None
    best_priority = float('-inf')
    while stack:
        u, p = stack.pop()
        if u in visited:
            continue
        visited.add(u)
        # Update the best state if necessary
        if p > best_priority:
            best_state = u
            best_priority = p
        # Explore the neighbors of u
        if u in g.AL:
            for v, w in g.AL[u]:
                if v not in visited:
                    stack.append((v, p + w + g.H[v]))
    return (best_state, best_priority)
    # '''
    # return a tuple (u, p) in which
    #   u: the local maximum state
    #   p: the priority/weight/desirability/cost of u

    # Note: weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    # '''
    # return (None, None)

In [44]:
class LocalBeamSearch(LocalSearchStrategy):
  def __init__(self, n: int):
    self.n = n  # Beam width
  def search(self, g: Graph, src: int) -> tuple:
    visited = set()
    beams = [[(src, 0)] for _ in range(self.n)]
    best_state = None
    best_priority = float('-inf')
    while beams:
        next_beams = []
        for beam in beams:
            new_beam = []
            for u, p in beam:
                if u in visited:
                    continue
                visited.add(u)
                # Update the best state if necessary
                if p > best_priority:
                    best_state = u
                    best_priority = p
                # Explore the neighbors of u
                if u in g.AL:
                    for v, w in g.AL[u]:
                        if v not in visited:
                            new_beam.append((v, p + w + g.H[v]))
            # Sort the new beam by priority and keep the top n states
            new_beam.sort(key=lambda x: x[1], reverse=True)
            next_beams.append(new_beam[:self.n])
        beams = next_beams
    return (best_state, best_priority)
    # '''
    # return a tuple (u, p) in which
    #   u: the local maximum state
    #   p: the priority/weight/desirability/cost of u

    # Note:
    # - weight of a node u = path_cost to u + heuristic value of u (similar to A*)
    # - parameter n is provided in the constructor
    # '''
    # return (None, None)

In [45]:
class SimulatedAnnealingSearch(LocalSearchStrategy):
    def search(self, g: Graph, src: int) -> tuple:
        current_state = src
        current_priority = float('-inf')
        best_state = None
        best_priority = float('-inf')
        temperature = 1.0
        cooling_rate = 0.01
        while temperature > 0.1:
            for _ in range(100):
                neighbors = g.AL.get(current_state, [])
                if neighbors:
                    next_state = random.choice(neighbors)[0]
                    next_priority = g.H[next_state]
                    delta_priority = next_priority - current_priority
                    if delta_priority > 0 or math.exp(delta_priority / temperature) > random.random():
                        current_state = next_state
                        current_priority = next_priority
                if current_priority > best_priority:
                    best_state = current_state
                    best_priority = current_priority
            temperature *= 1 - cooling_rate
        return (best_state, best_priority)
    # '''
    # return a tuple (u, p) in which
    #   u: the local maximum state
    #   p: the priority/weight/desirability/cost of u

    # Note: schedule(t) = 1/(t^2) with t is the iteration step
    # '''
    # return (None, None)

# Evaluation

In [47]:
hcs = HillClimbingSearch()
lbs = LocalBeamSearch(0)
sas = SimulatedAnnealingSearch()

for stg in [hcs, lbs, sas]:
  print(stg)
  u, p = stg.search(g, 0)
  print(u, p)

6 41
None -inf
1 13


# Submission

*   Students download the notebook after completion
*   Rename the notebook in which inserting your student ID at the beginning. \
For example, **123456-lec06-LocalSearch-HW.ipynb**
*   Finally, submit the file